<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/A(10%EA%B0%95_RNN(if_you_want_you).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import tensorflow as tf

# 1. 데이터 준비
text = "if you want you"  # 학습할 텍스트
chars = sorted(set(text))  # 고유한 문자들
char_to_idx = {char: idx for idx, char in enumerate(chars)}  # 문자 -> 인덱스 변환
idx_to_char = {idx: char for idx, char in enumerate(chars)}  # 인덱스 -> 문자 변환

# 데이터 전처리 (입력 시퀀스와 출력 시퀀스 생성)
input_seq = [char_to_idx[char] for char in text[:-1]]  # 마지막 문자는 제외
output_seq = [char_to_idx[char] for char in text[1:]]  # 첫 번째 문자는 제외

# 입력과 출력을 (1, 시퀀스 길이) 형태로 변환
input_seq = np.array(input_seq).reshape(1, -1)
output_seq = np.array(output_seq).reshape(1, -1)

vocab_size = len(chars)  # 고유한 문자 개수
embedding_dim = 8        # 임베딩 차원 수
input_length = input_seq.shape[1]  # 입력 시퀀스 길이

# 2. RNN 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# 3. 학습
model.fit(input_seq, output_seq, epochs=100)

"""
1. predictions의 구조

일반적으로 predictions는 3D 텐서입니다:
predictions.shape=(batch size,sequence length,vocab size)
여기서:
첫 번째 차원 (batch size): 여러 샘플(입력 시퀀스)의 묶음.
두 번째 차원 (sequence length): 시퀀스 내 각 타임스텝.
세 번째 차원 (vocab size): 예측된 확률 분포(어휘 크기만큼의 벡터).

2. 인덱싱: predictions[0, -1, :]

0: 배치(batch)에서 첫 번째 샘플을 선택.
-1: 해당 시퀀스의 마지막 타임스텝을 선택.
:: 마지막 차원(확률 벡터)의 모든 값을 선택.
"""

"""
model(input_eval): 현재 입력 시퀀스를 기반으로 모델이 다음 문자의 확률 분포를 예측.
입력 형태: (배치 크기, 타임스텝) 또는 (배치 크기, 타임스텝, 특징 수)
출력 형태: (배치 크기, 타임스텝, vocab_size)

여기서 vocab_size는 어휘 크기.
predictions[0, -1, :]: 가장 마지막 타임스텝의 예측 확률 분포를 가져옴.
 - predictions[0]: 첫 번째 배치의 출력.
 - predictions[0, -1, :]: 마지막 타임스텝의 출력 (확률 분포).
 - np.argmax(...): 확률 분포에서 가장 높은 값을 가진 인덱스(예측된 문자 ID)를 선택.
"""

# 4. 예측
def generate_text(model, start_string, num_generate):
    # 입력 텍스트를 숫자로 변환
    input_eval = [char_to_idx[char] for char in start_string]
    input_eval = np.array(input_eval).reshape(1, -1)

    generated_text = start_string

    # 문자를 예측
    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = np.argmax(predictions[0, -1, :])

        # 예측한 문자를 기존 텍스트에 추가
        predicted_char = idx_to_char[predicted_id]
        generated_text += predicted_char
        print("generated_text=", generated_text)

        # 다음 입력 시퀀스를 생성 (문자 추가)
        input_eval = np.append(input_eval, [[predicted_id]], axis=1)

    return generated_text


# 5. 예시 텍스트 생성
print(generate_text(model, start_string="if you ", num_generate=10))

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 2.3162
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.2961
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.2761
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.2557
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.2343
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.2113
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 2.1864
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.1591
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.1293
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.0968
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.0620
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.0251
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 1.9862
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 1.9455
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 1.9032
Epoch 16/100
1/1 ━━━━